# Ejercicio: creando una red neuronal artificial sencilla con Keras

<img src="https://keras.io/img/logo.png">

En este ejercicio vamos a crear una primera red neuronal artificial con Keras, para resolver un problema sencillo de clasificación binaria.

## Guía general

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

También vamos a fijar las semillas aleatorias de numpy y tensorflow para obtener resultados reproducibles entre varias ejecuciones del notebook

In [ ]:
import numpy as np
import tensorflow as tf
np.random.seed(1)
tf.random.set_seed(2)

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Carga de datos

Vamos a utilizar un problema sencillo de clasificación binaria, que no requiera de un preprocesado de datos complejo: [detección de cáncer de mama en base a unas características precalculadas](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Este dataset está incluído entre los [datasets de juguete de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer).

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()

Podemos ver que en este problema contamos con 569 datos en total, cada uno de ellos de 30 variables explicativas, y que solo existen dos clases.

In [ ]:
print("Forma de los datos de entrada:", data.data.shape)
print("Categorías a clasificar:", set(data.target))

Ahora separamos un tercio de los datos como conjunto de test, empleando el método [train_test_split de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=42)

Vamos a estandarizar las variables de entrada para asegurar un buen funcionamiento de la red neuronal. Esto puede hacerse fácilmente con el [StandardScaler de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

## Perceptrón

Empezaremos resolviendo el problema con la red neuronal más sencilla posible: un perceptrón. Esto es básicamente una red neuronal sin capas ocultas, teniendo solo una capa de salida que recibe información directamente de las variables explicativas.

### Definiendo la red

Para construir una red en Keras primero debemos definir el tipo de arquitectura:

* **Sequential**: cada nueva capa se conecta a la capa declarada inmediatamente antes en la red, siguiendo una cadena.
* **Functional**: cada capa puede conectarse a la salida de cualquier otra capa declarada antes en la red, siempre y cuando no se formen ciclos.

Para este ejercicio será suficiente con una arquitectura de tipo Sequential.

In [ ]:
from tensorflow.keras import Sequential

perceptron = Sequential()

Una vez la red ha sido inicializada, podemos ir añadiendo las capas deseadas de manera iterativa. Para construir un perceptrón solo nos hará falta una capa "clásica" de pesos desde las entradas a la salida. En Deep Learning moderno este tipo de capas se llaman **Dense**, porque conectan todas las variables de entrada con todas las salidas de la capa.

In [ ]:
from tensorflow.keras.layers import Dense

Para crear una capa densa suele bastar con declarar el número de unidades de salida (o desde un punto de vista clásico, el número de neuronas). No obstante, en un modelo de tipo Sequential estamos obligados a declarar el número de variables de entrada a la red neuronal cuando creamos nuestra primera capa. En este problema tenemos 30 variables explicativas, y podemos representar la salida de la red como una única neurona que nos diga si el dato es de clase 0 o 1. Por tanto, nos basta con crear la capa Dense y añadirla al modelo de la siguiente manera.

In [ ]:
perceptron.add(Dense(1, input_dim=30))

Como el problema es de clasificación binaria, nos interesa limitar el rango de valores que puede generar la única neurona de salida de la red. Para ello, lo más conveniente es usar la función de activación sigmoidal, la cual podemos añadir de la siguiente manera.

In [ ]:
from tensorflow.keras.layers import Activation
perceptron.add(Activation('sigmoid'))

Con esto, la definición de la red está completa. Podemos confirmar que la hemos construído correctamente pidiendo a Keras un resumen del modelo.

In [ ]:
perceptron.summary()

### Compilando la red

Tras definir la red, debemos realizar la compilación de la misma. La compilación es un proceso automático que transforma la definición de la red en una formulación simbólica equivalente para la que pueden calcularse las derivadas, permitiendo así ejecutar el algoritmo de retropropagación (backpropagation). Los únicos parámetros que debemos especificar son la función de pérdida o error que la red debe minimizar, y el optimizador a usar durante el aprendizaje.

Dado que estamos tratando con un problema de clasificación binaria, la función de pérdida más adecuada es la **binary crossentropy**. En cuanto al optimizador, de momento utilizaremos el **Stochastic Gradient Descent**. Como parámetro opcional, solicitaremos que como métrica se nos informe de la **accuracy** durante el entrenamiento de la red.

In [ ]:
perceptron.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Entrenando la red

Ahora podemos invocar al método `fit` de la red, el encargado de ejecutar el proceso de entrenamiento. Se configura de la siguiente manera

In [ ]:
perceptron.fit(
    X_train, # Features de los datos de entrenamiento
    y_train, # Etiquetas de los datos de entrenamiento
    epochs=25, # Número de épocas: las pasadas que hará la red neuronal sobre nuestros datos
    verbose=2, # Nivel de verbosidad de los mensajes
)

Ahora que nuestra red está entrenada, podemos obtener predicciones para el conjunto de test como

In [ ]:
probs = perceptron.predict(X_test)

Las predicciones se obtienen como una matriz con forma `(n_datos, 1)`, que pueden interpretarse como las probabilidades de clase positiva de cada uno de los datos de test

In [ ]:
probs

También podemos medir directamente el porcentaje de aciertos de nuestra red sobre el test, usando el método **evaluate**

In [ ]:
score = perceptron.evaluate(X_test, y_test)
print("Test loss", score[0])
print("Test accuracy", score[1])

## Perceptrón multicapa

Pasando a los años 80, podemos mejorar nuestra red introduciendo **capas ocultas**. En Keras podemos hacer esto de forma sencilla introduciendo capas Dense adicionales. Por ejemplo, podemos crear una red con una capa oculta de 64 neuronas de la siguiente manera.

In [ ]:
pmc = Sequential()
pmc.add(Dense(64, input_dim=30, activation='sigmoid'))
pmc.add(Dense(1, activation='sigmoid'))

En esta ocasión hemos añadido las activaciones declarándolas en la propia capa Dense, de forma que la notación resulte más compacta. Nótese también que hemos puesto una activación de tipo sigmoidal en la capa oculta, para que la red pueda comportarse de manera no lineal. La red, por tanto, queda de la siguiente manera

In [ ]:
pmc.summary()

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila el perceptrón multicapa y entrénalo con los datos de train. ¿Qué nivel de acierto se obtiene al evaluarlo en test?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Es probable que añadir una capa oculta no haya mejorado los resultados en test. En las siguientes secciones iremos añadiendo algunas de las recetas de Deep Learning para mejorar el rendimiento de la red.
</font>

***

### Más épocas

Un punto distintivo del método `fit` de Keras es que no reinicia el estado de la red neuronal, sino que continua el entrenamiento. Este comportamiento es diferente al que podemos observar en otras librerías de machine learning más clásico como **scikit-learn**, donde el método `fit` reinicia el entrenamiento del modelo. De este modo, en Keras podemos volver a llamar a `fit` para continuar entrenando durante más épocas.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Realiza una nueva llamada a `fit` sobre la misma red `pmc`, y mide el acierto sobre el conjunto de test.
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

Observa que la red ya muestra una accuracy muy alta en la primera época. Esto es porque se está continuando el entrenamiento realizado en la celda anterior. En Keras solo podemos reiniciar el entrenamiento de una red si la volvemos a construir desde cero.

### Más capas

Podemos introducir en la red más de una capa oculta si la configuramos adecuadamente en Keras. Por ejemplo, si queremos introducir 3 capas ocultas cada una de ellas con 64 neuronas, lo haríamos de la siguiente manera

In [ ]:
pmc2 = Sequential()
pmc2.add(Dense(64, input_dim=30, activation='sigmoid'))
pmc2.add(Dense(64, activation='sigmoid'))
pmc2.add(Dense(64, activation='sigmoid'))
pmc2.add(Dense(1, activation='sigmoid'))

pmc2.summary()

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Qué nivel de acierto obtienes ahora?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Observa cómo va evolucionando la <b>loss</b> durante el aprendizaje, y compáralo con las redes anteriores. Añadir más capas ocultas sin controlar el desvanecimiento de gradientes puede provocar que la red aprenda muy lentamente.
</font>

***

## Evitando el desvanecimiento de gradientes

Si queremos garantizar que todas las capas de la red aprendan correctamente, necesitamos utilizar alguna medida para evitar el desvanecimiento de gradientes. Una medida simple pero bastante efectiva es sustituir la función de la activación de las capas ocultas por la función Rectificada Lineal (ReLU).

In [ ]:
pmc_relu = Sequential()
pmc_relu.add(Dense(64, input_dim=30, activation='relu'))
pmc_relu.add(Dense(64, activation='relu'))
pmc_relu.add(Dense(64, activation='relu'))
pmc_relu.add(Dense(1, activation='sigmoid'))

pmc_relu.summary()

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Han mejorado los resultados respecto a la red anterior con activaciones sigmoidales?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

## Controlando el sobreajuste

Si la diferencia entre el loss o la accuracy de training y test es grande, puede que nuestro modelo esté sobreajustando a los datos de entrenamiento. Podemos añadir a nuestra red un método para controlar el sobreajuste usando capas de Dropout. Por ejemplo, para añadir un Dropout del 50% a todas las capas ocultas podemos hacer lo siguiente

In [ ]:
from tensorflow.keras.layers import Dropout

pmc_dropout = Sequential()
pmc_dropout.add(Dense(64, input_dim=30, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(64, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(64, activation='relu'))
pmc_dropout.add(Dropout(0.5))
pmc_dropout.add(Dense(1, activation='sigmoid'))

pmc_dropout.summary()

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Compila y entrena esta nueva red. ¿Han mejorado los resultados?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE

## Optimizando mejor

Como última mejora, vamos a introducir el optimizador [Adam](https://keras.io/api/optimizers/adam/). Este optimizador hace un uso más eficiente de los gradientes calculados durante el backpropagation, por lo que con el mismo número de épocas suele conseguir mejores resultados. Para configurarlo basta con definir la red de la misma manera, e indicar `optimizer="adam"` en el paso de compilación.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Define una nueva red copiando el diseño de la red anterior, compílala usando Adam como optimizador, entrena y mide el acierto sobre el conjunto de test. ¿Has conseguido mejora?
</font>

***


In [ ]:
####### INSERT YOUR CODE HERE